<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

In [16]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy.optimize import root_scalar
from scipy.optimize import root
from scipy.special import expit, logit

In [17]:
p=Props(['Acetone','Ethanol', 'Isopropanol', 'Water'])

In [18]:
def gamma(x,T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G=np.exp(-p.NRTL_alpha*tau)
    xG = x@G
    xtauG_xG = (x@(tau*G))/xG
    return np.exp(xtauG_xG + x@((G*(tau - xtauG_xG[None,:])/xG[None,:]).T))

In [19]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [20]:
def bubbleP_NRTL(x, T):
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

def bubbleT_NRTL(x, P):

    def f(T):
        return bubbleP_NRTL(x,T)[0]-P

    #mole-fraction weighted boiling points of each component at P
    #boiling points determined via Clausius Clapeyron, using the Hvap at the normal bp
    #for each component.  p.Hvap returns the heat of vaporization of all components for each
    #temperature if an array of temperatures is given.
    Tguess=np.dot(x,1/(1/p.Tbn-np.log(P/101325)*R/np.diagonal(p.Hvap(p.Tbn))))

    T=root_scalar(f, x0=Tguess, method='secant').root
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)

    return T, Pi/P



In [21]:
z = np.array([0.1,0.2,0.3,0.4])
P = 3e5
R=8.314
bubbleT_NRTL(z,P)


(Array(377.80220917, dtype=float64),
 Array([0.25464361, 0.1853556 , 0.28067204, 0.27932875], dtype=float64))

In [22]:
def dewP_NRTL(y, T):
    def f(vec):
        P = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Pguess, xguess= dewP_ideal(z,T)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Pguess, logit(xguess)]
    res=root(f, v0)
    return res.x[0], expit(res.x[1:])

In [23]:
def dewT_NRTL(y, P):
    def f(vec):
        T = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Tguess, xguess= dewT_ideal(z,P)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Tguess, logit(xguess)]
    res=root(f, v0)
    return res.x[0], expit(res.x[1:])

In [24]:
dewP_NRTL(z, 373)

(215679.4631248687, array([0.02121585, 0.13299478, 0.17452966, 0.67125971]))

In [25]:
dewT_NRTL(z, 215679)

(372.99993756950886, array([0.02121583, 0.13299476, 0.17452962, 0.67125978]))

In [ ]:
def flash_idealPT(z, P, T):

    K=p.Pvap(T)/P
    def rachford(VF):
        return np.sum(z*(K-1)/(VF*(K-1) +1))

    res=root_scalar(rachford, bracket=(0,1))
    VF = res.root
    x=z/(1-VF + K *VF)
    y=K*x
    return x, y, VF

In [40]:
def flash_NRTL_PT(z, P, T, maxiter = 100, tol=1e-12):

    dewP, dewx = dewP_NRTL(z, T)
    bubbleP, bubbley = bubbleP_NRTL(z,T)

    xguess = (P-dewP)/(bubbleP-dewP) * (z - dewx) +  dewx

    for i in range(maxiter):
        K=gamma(xguess,T)*p.Pvap(T)/P

        def rachford(VF):
            return np.sum(z*(K-1)/(VF*(K-1) +1))

        res=root_scalar(rachford, bracket=(0,1))
        VF = res.root
        x=z/(1-VF + K *VF)
        if (np.linalg.norm(xguess-x)<tol):
            break
        xguess = x

    y=K*x
    return x, y, VF, i

In [41]:
T=373
dewP, dewx = dewP_NRTL(z, T)
bubbleP, bubbley = bubbleP_NRTL(z,T)
flash_NRTL_PT(z, 0.4*dewP + 0.6*bubbleP, T)

(Array([0.06063401, 0.19965072, 0.29814679, 0.44156848], dtype=float64),
 Array([0.18481184, 0.2007525 , 0.30399265, 0.31044301], dtype=float64),
 0.31701301061806086,
 20)

(Array([0.06848634, 0.17319791, 0.24981187, 0.50850388], dtype=float64),
 Array([0.05363716, 0.1946712 , 0.28588545, 0.46580618], dtype=float64),
 Array([0.16987698, 0.20803144, 0.32127311, 0.30081848], dtype=float64),
 0.39885506390553066)

In [30]:
z

array([0.1, 0.2, 0.3, 0.4])

In [32]:
P=2.3e5
(P-dewP)/(bubbleP-dewP) * (z - dewx) +  dewx

Array([0.04800604, 0.1557796 , 0.21719529, 0.57901907], dtype=float64)